# Preparing Variants from PharmGKB for variant annotation
Only run this when the pipeline for data processing has been completed.

In [1]:
import os
import sys
import pandas as pd

In [23]:
path = "/Users/annamontaner/Documents/BSC/scratch/cli79/cli79334/projects/other/pharmGKB"

vars = pd.read_csv(os.path.join(path,"input_data", "variant_assembly.csv"), header=0)

In [25]:
# Check data types
vars.dtypes

vid                 object
variant             object
gene                object
gid                 object
assembly            object
begin              float64
end_               float64
intronic_offset    float64
ref_allele          object
ref_hgvs            object
var_allele          object
var_hgvs            object
chr                 object
dtype: object

In [26]:
# Check how many rows...
print("Total rows: ", len(vars))
print("Gene missing: ", len(vars[vars["gid"].isna()]))
print("Assembly missing: ", len(vars[vars["assembly"].isna()]))
print("Begin missing: ", len(vars[vars["begin"].isna()]))
print("Ref Allele missing: ", len(vars[vars["ref_allele"].isna()]))
print("Var Allele missing: ", len(vars[vars["var_allele"].isna()]))

Total rows:  12674
Gene missing:  954
Assembly missing:  264
Begin missing:  264
Ref Allele missing:  264
Var Allele missing:  264


In [27]:
# Remove rows for which reference or alternative alleles are not available
vars = vars[~vars["ref_allele"].isna()]
vars = vars[~vars["var_allele"].isna()]
print(len(vars))

12410


In [28]:
# Remove rows with "-"" instead of A/C/G/T
vars = vars[~(vars["ref_allele"]=="-")]
vars = vars[~(vars["var_allele"]=="-")]
print(len(vars))

12319


In [29]:
print(len(vars[vars["var_hgvs"].isna()]))

93


In [30]:
# Keep only SNVs

# Gemma: use var_hgvs notation
snp = vars[vars["var_hgvs"].str.contains(">", na=False)]

# Anna: a) use ref and alt alleles' length and b) remove variants containing characters other than A/C/G/T
# a)
snvs_tmp = vars[vars['ref_allele'].str.len() == vars['var_allele'].str.len()]

# b)
allowed_values = ['A', 'C', 'G', 'T']
snvs = snvs_tmp[snvs_tmp['ref_allele'].str.match(f'^({"|".join(allowed_values)})$') &
               snvs_tmp['var_allele'].str.match(f'^({"|".join(allowed_values)})$')]

# Check differences between snp (GEMMA) and snvs_tmp (Anna)
merged = snvs_tmp.merge(snp, how='outer', indicator=True)
comparison1 = merged[merged['_merge'] == 'left_only']

"""
Some single-nucleotide changes don't have a proper var_hgvs annotation so
it's better to use the nucleotide length of reference and alt alleles instead.
E.g. 
12441	PA166161042	rs78365220	G6PD	PA28469	GRCh38	154535270.0	154535270.0	0.0	A	NC_000023.11:g.154535270=	C	NaN	[GRCh38]chrX
"""

# Check differences between snp (GEMMA) and snvs (Anna)
merged = snvs.merge(snp, how='outer', indicator=True)
comparison2 = merged[merged['_merge'] == 'left_only']

"""
Still some differences between snp and snvs due to bad hgvs_alt notation. 
Example of a variant filtered out in snp but present in snvs:
9934	PA166157504	rs72552784	ABCB1	PA267	GRCh38	87516598.0	87516598.0	0.0	C	NC_000007.14:g.87516598=	A	NC_000006.12:g.160139851_160139853del	[GRCh38]chr7	left_only
"""

'\nStill some differences between snp and snvs due to bad hgvs_alt notation. Example of a variant filtered out in snp but present in snvs:\n9934\tPA166157504\trs72552784\tABCB1\tPA267\tGRCh38\t87516598.0\t87516598.0\t0.0\tC\tNC_000007.14:g.87516598=\tA\tNC_000006.12:g.160139851_160139853del\t[GRCh38]chr7\tleft_only\n'

In [31]:
print("Rows vars: ", len(vars))
print("Rows snp: ", len(snp))
print("Rows snvs_tmp: ", len(snvs_tmp))
print("Rows snvs: ", len(snvs))

Rows vars:  12319
Rows snp:  11981
Rows snvs_tmp:  12164
Rows snvs:  12088


In [32]:
# Check variant types in snvs dataframe
print('Ref allele types: ',snvs['ref_allele'].unique())
print('Var allele types: ',snvs['var_allele'].unique())

Ref allele types:  ['C' 'T' 'G' 'A']
Var allele types:  ['T' 'A' 'C' 'G']


In [33]:
# Change some column types
snvs['begin'] = snvs['begin'].astype("int")
snvs['chr'] = snvs['chr'].astype("str")
snvs['chr'] = snvs['chr'].str.split("]").str[1].str.replace("chr","")

/var/folders/kt/qf89bw917mg3nhrtrkj4_lmc0000gn/T/ipykernel_3194/3509328262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snvs['begin'] = snvs['begin'].astype("int")
/var/folders/kt/qf89bw917mg3nhrtrkj4_lmc0000gn/T/ipykernel_3194/3509328262.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snvs['chr'] = snvs['chr'].astype("str")
/var/folders/kt/qf89bw917mg3nhrtrkj4_lmc0000gn/T/ipykernel_3194/3509328262.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [34]:
snvs

,vid,variant,gene,gid,assembly,begin,end_,intronic_offset,ref_allele,ref_hgvs,var_allele,var_hgvs,chr
0,PA166156302,rs1000002,ABCC5,PA395,GRCh38,183917980,183917980.0,0.0,C,NC_000003.12:g.183917980=,T,NC_000003.12:g.183917980C>T,3
1,PA166156746,rs1000113,IRGM,PA142671652,GRCh38,150860514,150860514.0,0.0,C,NC_000005.10:g.150860514=,T,NC_000005.10:g.150860514C>T,5
2,PA166195421,rs10006452,UGT2B7,PA361,GRCh38,69112090,69112090.0,0.0,T,NC_000004.12:g.69112090=,A,NC_000004.12:g.69112090T>A,4
3,PA166195421,rs10006452,UGT2B7,PA361,GRCh38,69112090,69112090.0,0.0,T,NC_000004.12:g.69112090=,C,NC_000004.12:g.69112090T>C,4
4,PA166177121,rs10007051,NaN,NaN,GRCh38,129244309,129244309.0,0.0,C,NC_000004.12:g.129244309=,T,NC_000004.12:g.129244309C>T,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12658,PA166155732,rs4663871,UGT1A9,PA419,GRCh38,233672941,233672941.0,0.0,G,NC_000002.12:g.233672941=,A,NC_000002.12:g.233673102C>T,2
12659,PA166155732,rs4663871,UGT1A9,PA419,GRCh38,233672941,233672941.0,0.0,G,NC_000002.12:g.233672941=,T,NC_000002.12:g.233672941G>A,2
12660,PA166155877,rs45554333,UGT1A9,PA419,GRCh38,233672932,233672932.0,0.0,C,NC_000002.12:g.233672932=,A,NC_000002.12:g.233672941G>T,2
12661,PA166155877,rs45554333,UGT1A9,PA419,GRCh38,233672932,233672932.0,0.0,C,NC_000002.12:g.233672932=,T,NC_000002.12:g.233672932C>A,2


In [35]:
# Build final dataframe
df_tmp=snvs.copy()
df_tmp = df_tmp.fillna(".")

In [36]:
# Build non-existent columns
df_tmp['ID'] = "."
df_tmp['QUAL'] = "."
df_tmp['FILTER'] = "NONE"

# Build info column
info = "PGKBVID="+df_tmp["vid"]+";RSID="+df_tmp['variant']+";GENE="+df_tmp['gene']+";PGKBGID="+df_tmp['gid']+";HGVS="+df_tmp['var_hgvs']
df_tmp['INFO'] = info

In [95]:
# Build info Gemma
"""
def build_info(row):
    info_parts = []
    if not pd.isna(row['vid']):
        info_parts.append('PGKBVID=' + row['vid'])
    if not pd.isna(row['variant']):
        info_parts.append('RSID=' + row['variant'])
    if not pd.isna(row['gene']):
        info_parts.append('GENE=' + row['gene'])
    if not pd.isna(row['gid']):
        info_parts.append('PGKBGID=' + row['gid'])
    if not pd.isna(row['var_hgvs']):
        info_parts.append('HGVS=' + row['var_hgvs'])

    return ';'.join(info_parts)

snp['INFO'] = snp.apply(build_info, axis=1)

snp["ID"]=""
snp["FILTER"]="NONE"
snp["QUAL"] = ""
"""

/tmp/ipykernel_17847/1235631419.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snp['INFO'] = snp.apply(build_info, axis=1)


In [37]:
# Rename columns 
df_tmp.rename(columns={"chr":"#CHROM", "ref_allele": "REF", "var_allele": "ALT", "begin": "POS"}, inplace = True)

In [38]:
df_tmp

,vid,variant,gene,gid,assembly,POS,end_,intronic_offset,REF,ref_hgvs,ALT,var_hgvs,#CHROM,ID,QUAL,FILTER,INFO
0,PA166156302,rs1000002,ABCC5,PA395,GRCh38,183917980,183917980.0,0.0,C,NC_000003.12:g.183917980=,T,NC_000003.12:g.183917980C>T,3,.,.,NONE,PGKBVID=PA166156302;RSID=rs1000002;GENE=ABCC5;...
1,PA166156746,rs1000113,IRGM,PA142671652,GRCh38,150860514,150860514.0,0.0,C,NC_000005.10:g.150860514=,T,NC_000005.10:g.150860514C>T,5,.,.,NONE,PGKBVID=PA166156746;RSID=rs1000113;GENE=IRGM;P...
2,PA166195421,rs10006452,UGT2B7,PA361,GRCh38,69112090,69112090.0,0.0,T,NC_000004.12:g.69112090=,A,NC_000004.12:g.69112090T>A,4,.,.,NONE,PGKBVID=PA166195421;RSID=rs10006452;GENE=UGT2B...
3,PA166195421,rs10006452,UGT2B7,PA361,GRCh38,69112090,69112090.0,0.0,T,NC_000004.12:g.69112090=,C,NC_000004.12:g.69112090T>C,4,.,.,NONE,PGKBVID=PA166195421;RSID=rs10006452;GENE=UGT2B...
4,PA166177121,rs10007051,.,.,GRCh38,129244309,129244309.0,0.0,C,NC_000004.12:g.129244309=,T,NC_000004.12:g.129244309C>T,4,.,.,NONE,PGKBVID=PA166177121;RSID=rs10007051;GENE=.;PGK...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12658,PA166155732,rs4663871,UGT1A9,PA419,GRCh38,233672941,233672941.0,0.0,G,NC_000002.12:g.233672941=,A,NC_000002.12:g.233673102C>T,2,.,.,NONE,PGKBVID=PA166155732;RSID=rs4663871;GENE=UGT1A9...
12659,PA166155732,rs4663871,UGT1A9,PA419,GRCh38,233672941,233672941.0,0.0,G,NC_000002.12:g.233672941=,T,NC_000002.12:g.233672941G>A,2,.,.,NONE,PGKBVID=PA166155732;RSID=rs4663871;GENE=UGT1A9...
12660,PA166155877,rs45554333,UGT1A9,PA419,GRCh38,233672932,233672932.0,0.0,C,NC_000002.12:g.233672932=,A,NC_000002.12:g.233672941G>T,2,.,.,NONE,PGKBVID=PA166155877;RSID=rs45554333;GENE=UGT1A...
12661,PA166155877,rs45554333,UGT1A9,PA419,GRCh38,233672932,233672932.0,0.0,C,NC_000002.12:g.233672932=,T,NC_000002.12:g.233672932C>A,2,.,.,NONE,PGKBVID=PA166155877;RSID=rs45554333;GENE=UGT1A...


In [39]:
df = df_tmp[["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]].sort_values(by=['#CHROM','POS'])

In [40]:
df

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
10406,1,964594,.,G,A,.,NONE,PGKBVID=PA166153787;RSID=rs7414551;GENE=KLHL17...
10407,1,964594,.,G,A,.,NONE,PGKBVID=PA166153787;RSID=rs7414551;GENE=PLEKHN...
2689,1,1582436,.,C,T,.,NONE,PGKBVID=PA166165326;RSID=rs146898897;GENE=.;PG...
2344,1,2016961,.,C,A,.,NONE,PGKBVID=PA166258301;RSID=rs13303344;GENE=.;PGK...
2345,1,2016961,.,C,T,.,NONE,PGKBVID=PA166258301;RSID=rs13303344;GENE=.;PGK...
...,...,...,...,...,...,...,...,...
449,X,154536002,.,C,T,.,NONE,PGKBVID=PA166157859;RSID=rs1050828;GENE=IKBKG;...
12462,X,154536032,.,C,T,.,NONE,PGKBVID=PA166161038;RSID=rs137852315;GENE=G6PD...
12467,X,154536156,.,A,G,.,NONE,PGKBVID=PA166161036;RSID=rs76645461;GENE=G6PD;...
12468,X,154536168,.,G,C,.,NONE,PGKBVID=PA166157890;RSID=rs78478128;GENE=G6PD;...


In [48]:
print("Variants in unfiltered PharmGKB dataframe: ",len(vars['begin'].unique()))
print("Variants in filtered PharmGKB dataframe: ",len(df['POS'].unique()))

Variants in unfiltered PharmGKB dataframe:  6907
Variants in filtered PharmGKB dataframe:  6867


In [41]:
df.to_csv(os.path.join(path,"output_data","variant_assembly_auto.tsv"), sep="\t", header=True, index=False)

In [42]:
output_VCF = os.path.join(path,"output_data","variant_assembly_auto.vcf")
input_TSV = os.path.join(path,"output_data","variant_assembly_auto.tsv")
I = open(input_TSV, "r")
O = open(output_VCF, "w")

O.write("##fileformat=VCFv4.0\n")
O.write("##FILTER=<ID=NONE,Description=\"Not used\">\n")
O.write("##INFO=<ID=PGKBVID,Number=1,Type=String,Description=\"PharmGKB variant id\">\n")
O.write("##INFO=<ID=RSID,Number=1,Type=String,Description=\"dbSnp id\">\n")
O.write("##INFO=<ID=GENE,Number=1,Type=String,Description=\"Gene name\">\n")
O.write("##INFO=<ID=PGKBGID,Number=1,Type=String,Description=\"PharmGKB gene id\">\n")
O.write("##INFO=<ID=HGVS,Number=1,Type=String,Description=\"HGVS id\">\n")
O.write("##FORMAT=<ID=NO,Number=1,Type=String,Description=\"Not used\">\n")

for iLine in I:
	list = iLine.split("\t")
	newLine = '\t'.join(list)
	O.write(newLine)

I.close()
O.close()